# <center>Search

# BFS

## 994. Rotting Oranges (easy)

In a given grid, each cell can have one of three values:

- the value ```0``` representing an empty cell;
- the value ```1``` representing a fresh orange;
- the value ```2``` representing a rotten orange.

Every minute, any fresh orange that is adjacent (4-directionally) to a rotten orange becomes rotten.

Return the minimum number of minutes that must elapse until no cell has a fresh orange.  If this is impossible, return ```-1``` instead

In [ ]:
// BFS搜索，第一次将开始所有腐烂的橘子入队，每一轮将腐烂橘子的相邻位置入队，直到队空，最后判断是否还有好橘子

public int orangesRotting(int[][] grid) {
    Queue<Orange> q = new LinkedList<>();
    int m = grid.length;
    int n = grid[0].length;
    boolean noGoodOrange = true;  // 用来标记有没有好橘子
    for(int i = 0; i < m; i++){
        for(int j = 0; j < n; j++){
            if(grid[i][j] == 2){
                q.offer(new Orange(i, j));  //烂橘子入队
            } else if(grid[i][j] == 1 && noGoodOrange){  // 有好橘子,noGoodOrange置为false
                noGoodOrange = false;
            }
        }
    }
    if(q.isEmpty()){  // 队列为空有两种情况：1.全是好橘子,应返回-1；2.没有橘子,应返回0
        if(noGoodOrange) return 0; // noGoodOrange为true，说明没有好橘子，即没有橘子，返回0
        else return -1;            // noGoodOrange为false，说明有好橘子，那么一定全是好橘子,返回-1
    }
    int count = -1;  // 初始次数算第0次，所以从-1开始计数
    while(!q.isEmpty()){
        count++;
        int size = q.size();
        for(int k = 0; k < size; k++){
            Orange cur = q.poll();
            int i = cur.i;
            int j = cur.j;
            if(i - 1 >= 0 && grid[i - 1][j] == 1){ // 左边
                grid[i - 1][j] = 2;
                q.offer(new Orange(i - 1, j));
            }
            if(i + 1 < m && grid[i + 1][j] == 1){ // 右边
                grid[i + 1][j] = 2;
                q.offer(new Orange(i + 1, j));
            }
            if(j - 1 >= 0 && grid[i][j - 1] == 1){ // 上边
                grid[i][j - 1] = 2;
                q.offer(new Orange(i, j - 1));
            }
            if(j + 1 < n && grid[i][j + 1] == 1){ // 下边
                grid[i][j + 1] = 2;
                q.offer(new Orange(i, j + 1));
            }
        }
    }
    for(int i = 0; i < m; i++){     // 最后判断整个数组还有没有好橘子
        for(int j = 0; j < n; j++){
            if(grid[i][j] == 1)
                return -1;
        }
    }
    return count;
}

class Orange{
    int i;
    int j;

    public Orange(int i, int j){
        this.i = i;
        this.j = j;
    }
}

# Binary

## 33. Search in Rotated Sorted Array (medium)

Suppose an array sorted in ascending order is rotated at some pivot unknown to you beforehand.

(i.e., ```[0,1,2,4,5,6,7]``` might become ```[4,5,6,7,0,1,2]```).

You are given a target value to search. If found in the array return its index, otherwise return -1.

You may assume no duplicate exists in the array.

Your algorithm's runtime complexity must be in the order of ```O(log n)```.

In [1]:
// 思路一：先通过二分查找出数组中最小值的下标，即为旋转点，再根据target的大小确定去旋转点的左右进行二分查找
// 注意点：1.找旋转点的方法，如果循环中没有找到，则返回0，包含整个数组都旋转的情况（等于没有旋转）
//         2.判断该去旋转点的哪边搜索：如果旋转点值是0，说明数组没有旋转，对整个数组进行搜索；
//                                     否则，因为旋转点左边的值都大于它自己,所以如果【target < nums[0]】,说明target应在旋转点右边或不存在

 public int search(int[] nums, int target) {
    if(nums.length < 1)
            return -1;

    int left = 0;
    int right = nums.length - 1;
    int rotatedIndex = findRotatedIndex(nums, left, right); // 找旋转点
    
    if(target == nums[rotatedIndex])
        return rotatedIndex;
    
    if(rotatedIndex == 0)     // !!!旋转点是0的情况,需特殊考虑
        return binarySearch(nums, left, right, target);
    
    if(target < nums[0])
        return binarySearch(nums, rotatedIndex + 1, right, target);      
    
    return binarySearch(nums, left, rotatedIndex - 1, target);      
}

// 找旋转点，旋转点一定比它前面的值小
private int findRotatedIndex(int[] nums, int left, int right){
    if(nums[left] <= nums[right])
        return 0;
        
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(nums[mid] > nums[mid + 1]){
            return mid + 1;
        }
        if(nums[left] <= nums[mid]){
            left = mid + 1;
        } else {
            right = mid - 1;
        }
    }
    return 0;
}

// 单纯的二分查找
private int binarySearch(int[] nums, int left, int right, int target){
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(target == nums[mid])
            return mid;
        else if(target < nums[mid]){
            right = mid - 1;
        } else {
            left = mid + 1;
        }
    }
    return -1;
}

In [2]:
// 思路二：将数组一分为二，其中一定有一个是有序的，另一个可能是有序，也能是部分有序。此时有序部分用二分法查找
//         无序部分再一分为二，其中一个一定有序，另一个可能有序，可能无序。就这样循环.

public int search2(int[] nums, int target) {
    return helper(nums, 0, nums.length - 1, target);
}

private int helper(int[] nums, int left, int right, int target){
    if(left > right){
        return -1;
    }
    int mid = left + (right - left) / 2;
    int res = -1;
    if(nums[left] <= nums[mid]){  // left < mid 说明左边部分是有序的
        res = binarySearch(nums, left, mid, target);
        if(res != -1){
            return res; // 对左半二分查找，如果找到直接返回
        } else {        // 如果没有找到，则递归对右半部分进行操作
            return helper(nums, mid + 1, right, target);
        }
    } else {
        res = binarySearch(nums, mid + 1, right, target);
        if(res != -1){
            return res;
        } else {
            return helper(nums, left, mid, target);
        }
    }
}

private int binarySearch(int[] nums, int left, int right, int target){       
    while(left <= right){
        int mid = left + (right - left) / 2;
        if(target < nums[mid]){
            right = mid - 1;
        } else if(target > nums[mid]){
            left = mid + 1;
        } else{
            return mid;
        }
    }
    return -1;
}

// 思路类似，不用递归，更简洁
public int search3(int[] nums, int target) {
    int left = 0;
    int right = nums.length - 1;
    int mid = left + (right - left) / 2;
    while(left <= right){
        if(target == nums[mid])
            return mid;
            
        if(nums[left] <= nums[mid]){
            if(target >= nums[left] && target <= nums[mid]){
                right = mid - 1;
            } else {
                left = mid + 1;
            }
        } else {
            if(target >= nums[mid + 1] && target <= nums[right]){
                left = mid + 1;
            } else {
                right = mid - 1;
            }
        }
        mid = left + (right - left) / 2;
    }
    return -1;
}